In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
import gstools as gs
import matplotlib.pyplot as plt
import ee
import geemap
import requests
from shapely.geometry import Polygon
from shapely.geometry import Point
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

ee.Initialize()

In [5]:
fronterasMaritimasCol = ee.FeatureCollection('projects/ee-jolejua/assets/EEZ_land_union_v3_202003')
fronterasMaritimasCol = fronterasMaritimasCol.filter(ee.Filter.eq('UNION', 'Colombia'))
RegionesCol = ee.FeatureCollection('projects/ee-hides/assets/zonas_colombia')
world = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") # Filto de los anteriores datos para Colombia
ColombiaTerrestre = world.filter(ee.Filter.eq('ADM0_NAME', 'Colombia')) # Datos de fronteras a nivel departamental

In [7]:
#DATOS DEL SATELITE
satelite = "MODIS/061/MOD13Q1"
layer = 'NDVI'

# Intervalo de tiempo de interes
initDate = '2018-01-01'
finalDate = '2018-12-31'

dataset = ee.ImageCollection(satelite).filter(ee.Filter.date(initDate, finalDate))
ndvi = dataset.select(layer)

# Recorte de los datos usando el FeatureCollection
ndvi_clipped = ndvi.map(lambda image: image.clip(fronterasMaritimasCol.geometry()))

In [9]:
#parametros de visualización para plotear

ndviVis = {
  'min': 0,
  'max': 8000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
}


Map = geemap.Map(center=(4.6,-74),zoom=5)
Map.addLayer(ndvi_clipped, ndviVis, 'NDVI-Colombia')
Map.add_colorbar(vis_params=ndviVis, label='NDVI', layer_name='NDVI-Colombia')
# Se cre el mapa y se le agrega la capa para que se muestre

Map

Map(center=[4.6, -74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [17]:
Map_landCover = geemap.Map(center=(4.6,-74),zoom=5)
landCovel_col = ee.FeatureCollection('projects/ee-hides/assets/cobertura_tierra_clc_2018')
Map_landCover.addLayer(landCovel_col)
Map_landCover

Map(center=[4.6, -74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [20]:
landCovel_col.getInfo([0])


TypeError: getInfo() takes 1 positional argument but 2 were given